In [1]:
import pandas as pd
import numpy as np
from sklearn.neighbors import NearestNeighbors
from sklearn.feature_extraction.text import TfidfVectorizer

from geopy.geocoders import Nominatim

import re
import string
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords, wordnet as wn
from nltk.stem import PorterStemmer, WordNetLemmatizer

import Levenshtein as lev
import math
from collections import Counter

from sklearn.preprocessing import RobustScaler
from sklearn.model_selection import cross_val_score

from xgboost import XGBClassifier

In [2]:
train = pd.read_csv('../input/foursquare-location-matching/train.csv')
train = train.head()
train.head()

,id,name,latitude,longitude,address,city,state,zip,country,url,phone,categories,point_of_interest
0,E_000001272c6c5d,Café Stad Oudenaarde,50.859975,3.634196,Abdijstraat,Nederename,Oost-Vlaanderen,9700,BE,NaN,NaN,Bars,P_677e840bb6fc7e
1,E_000002eae2a589,Carioca Manero,-22.907225,-43.178244,NaN,NaN,NaN,NaN,BR,NaN,NaN,Brazilian Restaurants,P_d82910d8382a83
2,E_000007f24ebc95,ร้านตัดผมการาเกด,13.780813,100.484900,NaN,NaN,NaN,NaN,TH,NaN,NaN,Salons / Barbershops,P_b1066599e78477
3,E_000008a8ba4f48,Turkcell,37.844510,27.844202,Adnan Menderes Bulvarı,NaN,NaN,NaN,TR,NaN,NaN,Mobile Phone Shops,P_b2ed86905a4cd3
4,E_00001d92066153,Restaurante Casa Cofiño,43.338196,-4.326821,NaN,Caviedes,Cantabria,NaN,ES,NaN,NaN,Spanish Restaurants,P_809a884d4407fb


In [3]:
train.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5 entries, 0 to 4
Data columns (total 13 columns):
 #   Column             Non-Null Count  Dtype  
---  ------             --------------  -----  
 0   id                 5 non-null      object 
 1   name               5 non-null      object 
 2   latitude           5 non-null      float64
 3   longitude          5 non-null      float64
 4   address            2 non-null      object 
 5   city               2 non-null      object 
 6   state              2 non-null      object 
 7   zip                1 non-null      object 
 8   country            5 non-null      object 
 9   url                0 non-null      object 
 10  phone              0 non-null      object 
 11  categories         5 non-null      object 
 12  point_of_interest  5 non-null      object 
dtypes: float64(2), object(11)
memory usage: 648.0+ bytes


In [4]:
pairs = pd.read_csv('../input/foursquare-location-matching/pairs.csv')
#pairs = pairs.iloc[0:1000,:]
pairs.head()

,id_1,name_1,latitude_1,longitude_1,address_1,city_1,state_1,zip_1,country_1,url_1,...,longitude_2,address_2,city_2,state_2,zip_2,country_2,url_2,phone_2,categories_2,match
0,E_000001272c6c5d,Café Stad Oudenaarde,50.859975,3.634196,Abdijstraat,Nederename,Oost-Vlaanderen,9700,BE,NaN,...,3.635206,NaN,NaN,NaN,NaN,BE,NaN,NaN,Bars,True
1,E_000008a8ba4f48,Turkcell,37.844510,27.844202,Adnan Menderes Bulvarı,NaN,NaN,NaN,TR,NaN,...,27.845560,batı aydın,aydın,NaN,67500,TR,NaN,NaN,Electronics Stores,False
2,E_000023d8f4be44,Island Spa,14.518970,121.018702,"5th Flr, Newport Mall, Resorts World Manila",Pasay City,Metro Manila,NaN,PH,NaN,...,121.019827,NaN,NaN,NaN,NaN,PH,NaN,NaN,Spas,True
3,E_00007dcd2bb53f,TOGO'S Sandwiches,38.257797,-122.064599,"1380 Holiday Ln., Ste. B",Fairfield,CA,94534,US,https://locations.togos.com/ll/US/CA/Fairfield...,...,-122.064606,NaN,Fairfield,CA,NaN,US,NaN,NaN,Sandwich Places,True
4,E_0000c362229d93,Coffee Cat,7.082218,125.610244,F. Torres St.,Davao City,Davao Region,8000,PH,NaN,...,125.611457,E. Jacinto Extension,Davao City,Davao Region,8000,PH,NaN,NaN,"Coffee Shops, Cafés, Dessert Shops",False


In [5]:
pairs.shape

(578907, 25)

In [6]:
Latitude = "37.844510"
Longitude = "27.844202"
geolocator = Nominatim(user_agent="geoapiExercises")
location = geolocator.reverse(Latitude+","+Longitude)
address = location.raw['address']
print(address)

{'road': 'Adnan Menderes Bulvarı', 'suburb': 'Kurtuluş Mah.', 'city_district': 'Kurtuluş Mahallesi', 'town': 'Efeler', 'province': 'Aydın', 'ISO3166-2-lvl4': 'TR-09', 'region': 'Ege Bölgesi', 'postcode': '09110', 'country': 'Türkiye', 'country_code': 'tr'}


In [7]:
Latitude = "14.518970"
Longitude = "121.018702"
geolocator = Nominatim(user_agent="geoapiExercises")
location = geolocator.reverse(Latitude+","+Longitude)
address = location.raw['address']
print(address)

{'tourism': 'Hilton Manila', 'house_number': '1', 'road': 'Newport Boulevard', 'commercial': 'Resorts World Manila Complex', 'suburb': 'Barangay 183', 'city_district': 'Zone 20', 'city': 'Pasay', 'municipality': 'District 1', 'county': 'Pasay', 'state_district': 'Southern Manila District', 'region': 'Metro Manila', 'ISO3166-2-lvl3': 'PH-00', 'postcode': '1309', 'country': 'Pilipinas / Philippines', 'country_code': 'ph'}


In [8]:
Latitude = "7.082218"
Longitude = "125.610244"
geolocator = Nominatim(user_agent="geoapiExercises")
location = geolocator.reverse(Latitude+","+Longitude)
address = location.raw['address']
print(address)

{'amenity': 'Shabu Shabu', 'road': 'Sputnik Street', 'quarter': '19-B Garcia Heights', 'residential': 'Doña Vicenta Subdivision', 'suburb': 'Poblacion', 'city': 'Davao City', 'region': 'Davao Region', 'ISO3166-2-lvl3': 'PH-11', 'postcode': '8016', 'country': 'Pilipinas / Philippines', 'country_code': 'ph'}


In [9]:
address.get('city')

'Davao City'

In [10]:
loc_name_1 = ['city_1','state_1','zip_1']
loc_name_2 = ['city_2','state_2','zip_2']
name = ['city','state','postcode']

def find_loc(lat,lon,name):
    geolocator = Nominatim(user_agent="geoapiExercises")
    lat = str(lat); lon = str(lon)
    location = geolocator.reverse(lat+","+lon)
    address = location.raw['address']
    return address.get(name)
    
def find_loc_all(df,lat,lon,loc_name,name):

    for i in range(len(loc_name)):
        new_col = df.apply(lambda x: find_loc(x[lat],x[lon],name[i]), axis = 1)
        df[loc_name[i]].fillna(new_col, inplace=True)
    return df

In [11]:
# impute data base on location
# pairs = find_loc_all(pairs,'latitude_1','longitude_1',loc_name_1,name)
# pairs = find_loc_all(pairs,'latitude_2','longitude_2',loc_name_2,name)
# pairs.head()

In [12]:
pairs.iloc[:,0:12]

,id_1,name_1,latitude_1,longitude_1,address_1,city_1,state_1,zip_1,country_1,url_1,phone_1,categories_1
0,E_000001272c6c5d,Café Stad Oudenaarde,50.859975,3.634196,Abdijstraat,Nederename,Oost-Vlaanderen,9700,BE,NaN,NaN,Bars
1,E_000008a8ba4f48,Turkcell,37.844510,27.844202,Adnan Menderes Bulvarı,NaN,NaN,NaN,TR,NaN,NaN,Mobile Phone Shops
2,E_000023d8f4be44,Island Spa,14.518970,121.018702,"5th Flr, Newport Mall, Resorts World Manila",Pasay City,Metro Manila,NaN,PH,NaN,NaN,Spas
3,E_00007dcd2bb53f,TOGO'S Sandwiches,38.257797,-122.064599,"1380 Holiday Ln., Ste. B",Fairfield,CA,94534,US,https://locations.togos.com/ll/US/CA/Fairfield...,7074394747,Sandwich Places
4,E_0000c362229d93,Coffee Cat,7.082218,125.610244,F. Torres St.,Davao City,Davao Region,8000,PH,NaN,NaN,"Cafés, Coffee Shops"
...,...,...,...,...,...,...,...,...,...,...,...,...
578902,E_ffff80f94b2fee,Blk 332 Hougang Avenue 5,1.368397,103.898145,332 Hougang Ave 5,Singapore,Singapore,530332,SG,NaN,NaN,Residential Buildings (Apartments / Condos)
578903,E_ffff8cc1b92ab3,Fawsley Hall,52.206426,-1.178646,Fawsley Nr Daventry,Daventry,Northants,NN11 3BA,GB,NaN,NaN,Hotels
578904,E_ffff989ae206f8,Cricket Wireless Authorized Retailer,36.130623,-86.715636,825C Murfreesboro Pike,Nashville,TN,37217,US,https://www.cricketwireless.com/stores/ll/US/T...,+16156799940,Mobile Phone Shops
578905,E_ffffca745329ed,焼肉 和家,35.710712,139.775000,上野6-13-6,Taitō,東京都,110-0005,JP,NaN,338362989,BBQ Joints


In [13]:
pairs.iloc[:,12:]

,id_2,name_2,latitude_2,longitude_2,address_2,city_2,state_2,zip_2,country_2,url_2,phone_2,categories_2,match
0,E_da7fa3963561f8,Café Oudenaarde,50.869358,3.635206,NaN,NaN,NaN,NaN,BE,NaN,NaN,Bars,True
1,E_3f12df0b045062,Turkcell,37.838020,27.845560,batı aydın,aydın,NaN,67500,TR,NaN,NaN,Electronics Stores,False
2,E_12453effe251db,Island Spa Theater,14.518330,121.019827,NaN,NaN,NaN,NaN,PH,NaN,NaN,Spas,True
3,E_f131dcb7f07be9,Togo's,38.257938,-122.064606,NaN,Fairfield,CA,NaN,US,NaN,NaN,Sandwich Places,True
4,E_d52c3c4d53831c,Coffeecat,7.076016,125.611457,E. Jacinto Extension,Davao City,Davao Region,8000,PH,NaN,NaN,"Coffee Shops, Cafés, Dessert Shops",False
...,...,...,...,...,...,...,...,...,...,...,...,...,...
578902,E_75feaa1e0321cc,Blk332 Carpark,1.367956,103.897452,Hougang ave 5,NaN,NaN,NaN,SG,NaN,NaN,NaN,True
578903,E_ad351ad024c4dd,The Spa At Fawsley Hall,52.206974,-1.179698,NaN,NaN,NaN,NaN,GB,NaN,NaN,Gyms or Fitness Centers,False
578904,E_a5bc397a4eaeff,Cricket Wireless Authorized Retailer,36.134653,-86.723855,825C Murfreesboro Pike,Nashville,TN,37217,US,https://www.cricketwireless.com,+16156799940,Mobile Phone Shops,True
578905,E_04988888cfff60,Nagomiya,35.710412,139.774890,NaN,NaN,NaN,NaN,JP,NaN,NaN,BBQ Joints,True


In [14]:
pairs.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 578907 entries, 0 to 578906
Data columns (total 25 columns):
 #   Column        Non-Null Count   Dtype  
---  ------        --------------   -----  
 0   id_1          578907 non-null  object 
 1   name_1        578907 non-null  object 
 2   latitude_1    578907 non-null  float64
 3   longitude_1   578907 non-null  float64
 4   address_1     475383 non-null  object 
 5   city_1        512928 non-null  object 
 6   state_1       452316 non-null  object 
 7   zip_1         359509 non-null  object 
 8   country_1     578899 non-null  object 
 9   url_1         231806 non-null  object 
 10  phone_1       270022 non-null  object 
 11  categories_1  562613 non-null  object 
 12  id_2          578907 non-null  object 
 13  name_2        578907 non-null  object 
 14  latitude_2    578907 non-null  float64
 15  longitude_2   578907 non-null  float64
 16  address_2     312497 non-null  object 
 17  city_2        367490 non-null  object 
 18  stat

In [15]:
pairs.match.value_counts()

True     398786
False    180121
Name: match, dtype: int64

number of False variable is not enough

# Data preprocessing & Feature transformation:

1. location (latitude, longtitude): finding the distance between two variables
2. word preprocessing: remove url symbol, lowercasing, number removing, punctuation removal, white spaces removal, stop words removal, stemming and lemmatization


In [16]:
# 1. location
def distance(lat1, lon1, lat2, lon2):
    R = 6373.0
    d_lon = lon2 - lon1; d_lat = lat2 - lat1
    a = (np.sin(d_lat/2)) ** 2 + np.cos(lat1) * np.cos(lat2) * (np.sin(d_lon/2)) ** 2
    c = 2 * np.arctan2(np.sqrt(a), np.sqrt(1-a))
    distance = R * c
    return distance

In [17]:
train_pairs = pd.DataFrame(pairs.match)

In [18]:
train_pairs['distance'] = distance(pairs.latitude_1,pairs.longitude_1,pairs.latitude_2,pairs.longitude_2)

In [19]:
# 2 word preprocessing

# lowercase
def lower(df, cols):
    for col in cols:
        df[col] = df[col].fillna('').apply(lambda x: x.lower())
    return df

# number removing
def num_remove(df, cols):
    for col in cols:
        df[col] = df[col].fillna('').apply(lambda x: re.sub(r'\d+', '', x))
    return df

# punctuation removal
def punc_remove(df,cols):
    for col in cols:
        df[col] = df[col].fillna('').apply(lambda x: x.translate(str.maketrans("","", string.punctuation)))
    return df

# white spaces removal
def space_remove(df,cols):
    for col in cols:
        df[col] = df[col].fillna('').apply(lambda x: x.strip())
    return df

# stop words removal

def list_to_string(lis):
    string = ''
    for i in lis:
        string += i
        string += ' '
    return string

def stop(string):
    stops = set(stopwords.words('english'))
    tokens = word_tokenize(string)
    result = [i for i in tokens if not i in stops]
    return result
    
def stop_remove(df,cols):
    for col in cols:
        df[col] = df[col].fillna('').apply(lambda x: stop(x))
    return df

# stemming and lemmatization
def lemma_stem(lis):
    stemmer = PorterStemmer()
    lemmatizer = WordNetLemmatizer()
    new_lis = []
    for word in lis:
        word_new = lemmatizer.lemmatize(word)
        word_new = stemmer.stem(word_new)
        new_lis += [word_new]
    return new_lis

def lemma_stemming(df,cols):
    for col in cols:
        df[col] = df[col].fillna('').apply(lambda x: lemma_stem(x))
    return df

In [20]:
# remove url
def remove_URL(df,cols):
    for col in cols:
        df[col] = df[col].fillna('').apply(lambda x: re.sub(r"http\S+", "", x))
    return df

In [21]:
word_columns = ['name_1','address_1','city_1','state_1','url_1','categories_1','name_2','address_2','city_2','state_2','url_2','categories_2']
url_columns = ['url_1','url_2']
pairs[url_columns] = remove_URL(pairs, url_columns)[url_columns]
pairs[word_columns] = lower(pairs, word_columns)[word_columns]
pairs[word_columns] = num_remove(pairs, word_columns)[word_columns]
pairs[word_columns] = punc_remove(pairs, word_columns)[word_columns]
pairs[word_columns] = space_remove(pairs, word_columns)[word_columns]
pairs[word_columns] = stop_remove(pairs, word_columns)[word_columns]
pairs[word_columns] = lemma_stemming(pairs, word_columns)[word_columns]

text similarity

In [22]:
def fuzzy_similarity(df, cols_1, cols_2):
    # length for cols_1 and cols_2 must be the same.
    for i in range(len(cols_1)):
        df[f"{cols_1[i]}_fuzzy"] = df.apply(lambda x: lev.ratio(x[cols_1[i]],x[cols_2[i]]), axis = 1)
    return df    

def counter_cosine_similarity(c1, c2):
    terms = set(c1).union(c2)
    dotprod = sum(c1.get(k, 0) * c2.get(k, 0) for k in terms)
    magA = math.sqrt(sum(c1.get(k, 0)**2 for k in terms))
    magB = math.sqrt(sum(c2.get(k, 0)**2 for k in terms))
    return dotprod / (magA * magB)

In [23]:
col_1 = ['name_1','address_1','city_1','state_1','url_1','categories_1']
col_2 = ['name_2','address_2','city_2','state_2','url_2','categories_2']
pairs_new = pairs.copy()
for col in word_columns:
    pairs_new[col] = pairs_new[col].apply(lambda x: list_to_string(x))
pairs_new = fuzzy_similarity(pairs_new, col_1,col_2)

In [24]:
pairs_new.head()

,id_1,name_1,latitude_1,longitude_1,address_1,city_1,state_1,zip_1,country_1,url_1,...,url_2,phone_2,categories_2,match,name_1_fuzzy,address_1_fuzzy,city_1_fuzzy,state_1_fuzzy,url_1_fuzzy,categories_1_fuzzy
0,E_000001272c6c5d,café stad oudenaard,50.859975,3.634196,abdijstraat,nederenam,oostvlaanderen,9700,BE,,...,,NaN,bar,True,0.857143,0.000000,0.0,0.0,1.0,1.000000
1,E_000008a8ba4f48,turkcel,37.844510,27.844202,adnan mender bulvarı,,,NaN,TR,,...,,NaN,electron store,False,1.000000,0.312500,0.0,1.0,1.0,0.437500
2,E_000023d8f4be44,island spa,14.518970,121.018702,th flr newport mall resort world manila,pasay citi,metro manila,NaN,PH,,...,,NaN,spa,True,0.733333,0.000000,0.0,0.0,1.0,1.000000
3,E_00007dcd2bb53f,togo sandwich,38.257797,-122.064599,holiday ln ste b,fairfield,ca,94534,US,,...,,NaN,sandwich place,True,0.526316,0.000000,1.0,1.0,1.0,1.000000
4,E_0000c362229d93,coffe cat,7.082218,125.610244,f torr st,davao citi,davao region,8000,PH,,...,,NaN,coffe shop café dessert shop,False,0.900000,0.444444,1.0,1.0,1.0,0.533333


In [25]:
import spacy
nlp = spacy.load("en_core_web_lg")
#nlp = spacy.load("en_core_web_md")


doc1 = nlp(u'the person wear red T-shirt')
doc2 = nlp(u'red T-shirt the person wear')
doc3 = nlp(u'the and are bad good')


print(doc1.similarity(doc2)) 
print(doc1.similarity(doc3))
print(doc2.similarity(doc3)) 

1.0000000610236768
0.6768339781706519
0.6768340415356061


In [26]:
# wordnet similarity method
def similarity(word1, word2):
    max_sim = []
    syn1 = wn.synsets(word1)
    syn2 = wn.synsets(word2)
    for s1 in syn1:
        for s2 in syn2:
            max_sim.append(s1.path_similarity(s2))
    max_sim = [z for z in max_sim if z!=None]
    if max_sim == []:
        return 0
    return max(max_sim)
    
def similarity_lis(lis_1, lis_2):
    score_lis = []
    for word in lis_1:
        score_lis2 = []
        for word_2 in lis_2:
            sim = similarity(word, word_2)
            if sim != 0:
                score_lis2 += [sim]
        if score_lis2 == []:
            score_lis += [0]
        else:
            score_lis+=[max(score_lis2)]
    if score_lis == []:
        return 0
    return sum(score_lis)/len(score_lis)

def sim_lis(lis_1, lis_2):
    return (similarity_lis(lis_1, lis_2) + similarity_lis(lis_2, lis_1))/2

def nltk_similarity(df, cols_1, cols_2):
    # length for cols_1 and cols_2 must be the same.
    for i in range(len(cols_1)):
        df[f"{cols_1[i]}_nltk"] = df.apply(lambda x: sim_lis(x[cols_1[i]],x[cols_2[i]]), axis = 1)
    return df   

In [27]:
col_1 = ['name_1','address_1','city_1','state_1','url_1','categories_1']
col_2 = ['name_2','address_2','city_2','state_2','url_2','categories_2']
pairs = nltk_similarity(pairs, col_1,col_2)

In [28]:
pairs.head()

,id_1,name_1,latitude_1,longitude_1,address_1,city_1,state_1,zip_1,country_1,url_1,...,url_2,phone_2,categories_2,match,name_1_nltk,address_1_nltk,city_1_nltk,state_1_nltk,url_1_nltk,categories_1_nltk
0,E_000001272c6c5d,"[café, stad, oudenaard]",50.859975,3.634196,[abdijstraat],[nederenam],[oostvlaanderen],9700,BE,[],...,[],NaN,[bar],True,0.000000,0.000000,0.0,0.0,0.0,1.000000
1,E_000008a8ba4f48,[turkcel],37.844510,27.844202,"[adnan, mender, bulvarı]",[],[],NaN,TR,[],...,[],NaN,"[electron, store]",False,0.000000,0.000000,0.0,0.0,0.0,0.461310
2,E_000023d8f4be44,"[island, spa]",14.518970,121.018702,"[th, flr, newport, mall, resort, world, manila]","[pasay, citi]","[metro, manila]",NaN,PH,[],...,[],NaN,[spa],True,0.875000,0.000000,0.0,0.0,0.0,1.000000
3,E_00007dcd2bb53f,"[togo, sandwich]",38.257797,-122.064599,"[holiday, ln, ste, b]",[fairfield],[ca],94534,US,[],...,[],NaN,"[sandwich, place]",True,0.766667,0.000000,0.0,1.0,0.0,1.000000
4,E_0000c362229d93,"[coffe, cat]",7.082218,125.610244,"[f, torr, st]","[davao, citi]","[davao, region]",8000,PH,[],...,[],NaN,"[coffe, shop, café, dessert, shop]",False,0.000000,0.127778,0.0,0.5,0.0,0.374359


In [29]:
pairs.columns

Index(['id_1', 'name_1', 'latitude_1', 'longitude_1', 'address_1', 'city_1',
       'state_1', 'zip_1', 'country_1', 'url_1', 'phone_1', 'categories_1',
       'id_2', 'name_2', 'latitude_2', 'longitude_2', 'address_2', 'city_2',
       'state_2', 'zip_2', 'country_2', 'url_2', 'phone_2', 'categories_2',
       'match', 'name_1_nltk', 'address_1_nltk', 'city_1_nltk', 'state_1_nltk',
       'url_1_nltk', 'categories_1_nltk'],
      dtype='object')

In [30]:
cols = ['latitude_1','longitude_1','country_1','latitude_2','longitude_2','country_2','name_1_nltk', 'address_1_nltk', 'city_1_nltk', 'state_1_nltk',
       'url_1_nltk', 'categories_1_nltk','match']
pairs[cols].head()

,latitude_1,longitude_1,country_1,latitude_2,longitude_2,country_2,name_1_nltk,address_1_nltk,city_1_nltk,state_1_nltk,url_1_nltk,categories_1_nltk,match
0,50.859975,3.634196,BE,50.869358,3.635206,BE,0.000000,0.000000,0.0,0.0,0.0,1.000000,True
1,37.844510,27.844202,TR,37.838020,27.845560,TR,0.000000,0.000000,0.0,0.0,0.0,0.461310,False
2,14.518970,121.018702,PH,14.518330,121.019827,PH,0.875000,0.000000,0.0,0.0,0.0,1.000000,True
3,38.257797,-122.064599,US,38.257938,-122.064606,US,0.766667,0.000000,0.0,1.0,0.0,1.000000,True
4,7.082218,125.610244,PH,7.076016,125.611457,PH,0.000000,0.127778,0.0,0.5,0.0,0.374359,False


In [31]:
X = pairs[cols]
X['country_same'] = np.where(X['country_1'] == X['country_2'],1,0)
del X['country_1'] 
del X['country_2']

/opt/conda/lib/python3.7/site-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  


In [32]:
col_scaler = ['latitude_1','longitude_1','latitude_2','longitude_2']
X_transform = X[col_scaler]
transformer = RobustScaler().fit(X_transform)
X[col_scaler] = transformer.transform(X_transform)

/opt/conda/lib/python3.7/site-packages/pandas/core/frame.py:3678: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self[col] = igetitem(value, i)


In [33]:
X.head()

,latitude_1,longitude_1,latitude_2,longitude_2,name_1_nltk,address_1_nltk,city_1_nltk,state_1_nltk,url_1_nltk,categories_1_nltk,match,country_same
0,0.432146,-0.160787,0.432051,-0.160824,0.000000,0.000000,0.0,0.0,0.0,1.000000,True,1
1,0.062999,-0.014337,0.062870,-0.014368,0.000000,0.000000,0.0,0.0,0.0,0.461310,False,1
2,-0.598564,0.549291,-0.597784,0.549274,0.875000,0.000000,0.0,0.0,0.0,1.000000,True,1
3,0.074721,-0.921160,0.074766,-0.921223,0.766667,0.000000,0.0,1.0,0.0,1.000000,True,1
4,-0.809486,0.577066,-0.808627,0.577050,0.000000,0.127778,0.0,0.5,0.0,0.374359,False,1


In [34]:
y = pairs['match']
xgb = XGBClassifier(nthread = -1)
cross_score = cross_val_score(xgb, X, y, cv=5, n_jobs = -1)
print(cross_score)
print(sum(cross_score)/5)

[1. 1. 1. 1. 1.]
1.0
